In [1]:
import scanpy as sc
import pandas as pd 
import numpy as np
import anndata
import re
import h5py
import scipy.sparse as scs
import concurrent.futures
import seaborn as sns
from scipy.sparse import csc_matrix
import random
random.seed(123)
from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:
from datetime import date
import hisepy
import os
import pandas
import tarfile

In [3]:
import session_info
session_info.show()

# Download metadata from HISE

In [4]:
import hisepy as hp 
meta_uuid = "9c417a2e-ee43-43b8-9f05-7940b7a42920"
label_file = hp.cache_files([meta_uuid])

downloading fileID: 9c417a2e-ee43-43b8-9f05-7940b7a42920
Files have been successfully downloaded!


In [5]:
meta_data = pd.read_csv("/home/jupyter/cache/9c417a2e-ee43-43b8-9f05-7940b7a42920/scRNA_meta_data-2024-05-09.csv")
meta_data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'lastUpdated', 'sample.id',
       'sample.bridgingControl', 'sample.sampleKitGuid', 'sample.visitName',
       'sample.visitDetails', 'sample.drawDate', 'sample.daysSinceFirstVisit',
       'file.id', 'file.name', 'file.batchID', 'file.panel', 'file.pool',
       'file.fileType', 'file.majorVersion', 'subject.id',
       'subject.biologicalSex', 'subject.birthYear', 'subject.ethnicity',
       'subject.partnerCode', 'subject.race', 'subject.subjectGuid',
       'cohort.cohortGuid', 'sample.diseaseStatesRecordedAtVisit',
       'pbmc_sample_id', 'subject.covidVaxDose1.daysSinceFirstVisit',
       'subject.covidVaxDose2.daysSinceFirstVisit', 'Covid_exclusion',
       'subjectGuid', 'CMV'],
      dtype='object')

In [55]:
#meta_data=pd.read_csv('/home/jupyter/IH-A-Aging-Analysis-Notebooks/Qiuyu-Notebooks/scRNA/hise_meta_data_2024-01-23_fixed.csv')
meta_data=meta_data[meta_data['Covid_exclusion']=='no']
TimePoints=['Flu Year 1 Day 0']
meta_data_subset=meta_data[meta_data['sample.visitName'].isin(TimePoints)]

In [56]:
len(meta_data_subset)

92

In [57]:
meta_data_subset["sample.visitName"].unique()

array(['Flu Year 1 Day 0'], dtype=object)

In [58]:
#meta_data=pd.read_csv('/home/jupyter/IH-A-Aging-Analysis-Notebooks/Qiuyu-Notebooks/scRNA/hise_meta_data_2024-01-23_fixed.csv')

In [59]:
from tqdm import tqdm
import os
import scanpy as sc

h5_list = []
folder_path = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/sample_h5ad/"

for pbmc_sample_id in tqdm(meta_data_subset["pbmc_sample_id"].tolist()):
    # Construct the new file name using the pbmc_sample_id
    file_name = f"{pbmc_sample_id}.h5ad"
    
    # Get all files in the folder
    file_list = os.listdir(folder_path)
    
    # Find the file matching the pattern
    matching_files = [file for file in file_list if file.startswith(pbmc_sample_id)]

    # Check if any matching file found
    if matching_files:
        file_path = os.path.join(folder_path, matching_files[0])
        try:
            result = sc.read_h5ad(file_path)
            h5_list.append(result)
            print()
        except Exception as e:
            print(f'Error reading {file_name}: {e}')
    else:
        print(f'No file found for {pbmc_sample_id}')


  1% 1/92 [00:00<00:25,  3.62it/s]

  2% 2/92 [00:00<00:24,  3.69it/s]

  3% 3/92 [00:00<00:24,  3.64it/s]

  4% 4/92 [00:01<00:24,  3.57it/s]

  5% 5/92 [00:01<00:25,  3.41it/s]

...

 95% 87/92 [00:29<00:01,  3.49it/s]

 97% 89/92 [00:29<00:00,  3.94it/s]

 98% 90/92 [00:29<00:00,  3.84it/s]

 99% 91/92 [00:30<00:00,  3.63it/s]

100% 92/92 [00:30<00:00,  3.03it/s]

In [41]:
# from tqdm import tqdm
# h5_list = []
# for i in tqdm(range(len(meta_data_subset["pbmc_sample_id"].tolist()))):
#     try:
#         result = sc.read_h5ad("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/sample_h5ad/"+meta_data_subset["pbmc_sample_id"].tolist()[i]+'.h5ad')
#         h5_list.append(result)
#         print()
#     except Exception as e:
#         print(f'Error reading {file_name}: {e}')

In [60]:
%%time
combined= anndata.concat(h5_list)

/opt/conda/lib/python3.10/site-packages/anndata/_core/merge.py:217: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(dtype):


CPU times: user 3min 14s, sys: 35.7 s, total: 3min 50s
Wall time: 3min 51s


In [61]:
%%time
combined.write_h5ad('/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/Object_generation/FluYr1D0_certpro.h5ad')

CPU times: user 9.49 s, sys: 26.5 s, total: 36 s
Wall time: 2min 37s


In [62]:
combined.obs.index=combined.obs['barcodes']

In [63]:
%%time
fitlered_gene=pd.DataFrame()
for i in combined.obs['AIFI_L3'].unique():
    print(i)
    adata_subset=combined[combined.obs['AIFI_L3']==i]
    sc.pp.filter_genes(adata_subset, min_cells=round(np.shape(adata_subset.X)[0]*0.1))
    gene_list=pd.DataFrame(list(adata_subset.var.index))
    gene_list.columns=['gene']
    gene_list['AIFI_L3']=i
    fitlered_gene=pd.concat([fitlered_gene,gene_list])

GZMB- CD27+ EM CD4 T cell


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


ISG+ CD14 monocyte


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


GZMK+ Vd2 gdT


...

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


BaEoMaP cell
KLRB1+ memory CD4 Treg
DN T cell


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-

CD4 MAIT
CMP cell
SOX4+ Vd1 gdT
CPU times: user 1min 11s, sys: 43.8 s, total: 1min 55s
Wall time: 1min 55s


/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: Fut

In [64]:
fitlered_gene.to_csv('/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/CertPro_analysis/scripts/Object_generation/filtered_gene_list_for_fluyr1D0_certpro.csv')